We feel that it is slightly problematic that there are lots of null values (recorded as zeros) corresponding to cat facial features not detected. 

In [18]:
import numpy as np
import csv 
x_vals = np.loadtxt("../data/regression_training/features_v2.txt")
x_vals = x_vals[:, [0, 1, 2, 3, 4, 6, 8, 9, 10]]
x_header = [
    'eye_ratio', 'head_size', 'ear_ratio', 'conf_head', 'conf_eye_0', 'conf_ear_0',
    'conf_nose', 'sharpness', 'head_distance',
]
with open("../data/regression_training/score_v2.csv") as csv_file: 
    reader = csv.reader(csv_file) 
    y_vals = np.array([int(row[-1]) for row in reader])
    y_vals = y_vals[y_vals != 0]
any_features = np.any(x_vals[:, 3:7] != 0, axis=1)
x_vals = x_vals[any_features]
y_vals = y_vals[any_features]

In [105]:
x_vals_all = np.loadtxt("../data/regression_training/features_v2.txt")[any_features]

In [19]:
x_vals = np.hstack((x_vals, np.sum(x_vals[:, 3:7] != 0, axis=1).reshape(-1, 1)))
x_header.append("num_features_detected")

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [42]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    # print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * (depth - 1)
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}{}".format(indent, tree_.value[node]))

    recurse(0, 1)

In [59]:
permute = np.random.permutation(len(y_vals))

In [52]:
for i in range(1, 15): 
    model_bin = DecisionTreeClassifier(max_depth=i) 
    model_bin = model_bin.fit(x_vals[permute][:180], y_vals[permute][:180] > 3) 
    print(f"Depth: {i}", model_bin.score(x_vals[permute][180:], y_vals[permute][180:] > 3)) 

Depth: 1 0.743801652892562
Depth: 2 0.7851239669421488
Depth: 3 0.7851239669421488
Depth: 4 0.768595041322314
Depth: 5 0.7355371900826446
Depth: 6 0.7107438016528925
Depth: 7 0.7768595041322314
Depth: 8 0.7520661157024794
Depth: 9 0.71900826446281
Depth: 10 0.7272727272727273
Depth: 11 0.71900826446281
Depth: 12 0.7355371900826446
Depth: 13 0.7355371900826446
Depth: 14 0.7272727272727273


In [68]:
model_bin = DecisionTreeClassifier(max_depth=2)
model_bin.fit(x_vals[permute][:180], y_vals[permute][:180] > 3)
tree_to_code(model_bin, x_header)

if num_features_detected <= 3.5:
  if conf_ear_0 <= 0.7005633413791656:
    [[55.  9.]]
  else:  # if conf_ear_0 > 0.7005633413791656
    [[9. 7.]]
else:  # if num_features_detected > 3.5
  if conf_eye_0 <= 0.8488049805164337:
    [[16. 15.]]
  else:  # if conf_eye_0 > 0.8488049805164337
    [[ 9. 60.]]


Number of features detected, confidence of ears and confidence of eyes consistently show up as the most important features. Perhaps we can improve the performance of any logistic regression model by first taking out some frames that are terrible.

In [83]:
model_bin = DecisionTreeClassifier(max_depth=1, criterion="entropy")
# Note the additional penalty for false positives
model_bin.fit(x_vals[permute][:180], y_vals[permute][:180] > 3)
tree_to_code(model_bin, x_header)

if num_features_detected <= 3.5:
  [[64. 16.]]
else:  # if num_features_detected > 3.5
  [[25. 75.]]


In [84]:
np.average((y_vals > 3) == (x_vals[:, -1] == 4))

0.7508305647840532

In [85]:
np.average(y_vals > 3)

0.4950166112956811

In [91]:
x_vals[:, :-1] @ np.loadtxt("../data/regression_parameter_results/log_params_any_features_v1.txt").T

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 11 is different from 9)

In [109]:
np.average((y_vals > 3)==(np.argmax(x_vals_all @ np.loadtxt("../data/regression_parameter_results/log_params_any_features_v1.txt").T, axis=1) > 3))

0.5913621262458472

In [103]:
(y_vals > 3).shape

(301,)

In [107]:
(np.argmax(x_vals_all @ np.loadtxt("../data/regression_parameter_results/log_params_any_features_v1.txt").T, axis=1) > 3).shape

(301,)

In [123]:
np.average((y_vals > 3) == np.logical_and(np.argmax(x_vals_all @ np.loadtxt("../data/regression_parameter_results/log_params_all_features_v1.txt").T, axis=1) > 3, x_vals[:, -1] == 4))

0.7508305647840532

(168, 11)